In [25]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd
import datetime

In [26]:
esp_exc = pd.read_excel("especialidades_exceptuadas.xlsx").esp_exceptuadas.tolist()

In [27]:
prestaciones_imagenes = pd.read_excel("prestaciones_imagenes.xlsx",index_col=0).prestac_pres_prestacion.tolist()
prestaciones_imagenes = [str(x) for x in prestaciones_imagenes]
prestaciones_imagenes = str(prestaciones_imagenes).strip("[").strip("]")

In [28]:
centros_de_imagenes = str(pd.read_excel("centros_de_imagenes.xlsx",index_col=0).id_pre_prestador.tolist()).strip("[").strip("]")

In [29]:
homologaciones = pd.read_excel("homologaciones.xlsx",dtype=str,index_col=0)

In [30]:
validacion_0 = homologaciones.validacion_0.dropna().tolist()
validacion_1 = homologaciones.validacion_1.dropna().tolist()
validacion_2 = homologaciones.validacion_2.dropna().tolist()
validacion_3 = homologaciones.validacion_3.dropna().tolist()

In [31]:
mayor = "2020-05"
menor = "2019-04"

In [32]:
df = querydbtopandas(f"""select a11.transac  transac,
a11.id_afiliado  id_afi_afiliado,
a11.id_pres_prestacion  id_pres_prestacion,
max(a17.desc_pres_prestacion)  desc_pres_prestacion,
max(a17.prestac_pres_prestacion)  prestac_pres_prestacion,
a11.transac_rela  transac_rela,
a12.id_pre_prestador  id_pre_prestador0,
max(a14.desc_pre_nombre)  desc_pre_nombre0,
a11.fecha,
Q.deno,
sum(a11.canti)  QTRANSAC
from DBA.ft_cm_transac_remotas a11
join DBA.d_pre_demandante_transac a12
 on (a11.id_pre_prestador_demandante = a12.id_pre_prestador_demandante)
join DBA.d_pre_prestador a14
 on (a12.id_pre_prestador = a14.id_pre_prestador)
join DBA.d_pres_prestacion a17
 on (a11.id_pres_prestacion = a17.id_pres_prestacion)
LEFT JOIN dba.prestad_costo_centros E
    ON a12.id_pre_prestador_demandante = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
LEFT JOIN dba.costo_centros Q
    ON E.centro = Q.centro

where (cast(a11.fecha as date) >= date("{menor}") and cast(a11.fecha as date) < date("{mayor}")
 and a11.id_pre_prestador in ({centros_de_imagenes})
 and a11.tran_tipo in ("IP")
 and a11.baja_fecha is null
 and recha in (0)
 and RTRIM(a17.prestac_pres_prestacion) in ({prestaciones_imagenes})
 and a12.id_pre_prestador <> 0)
group by a11.transac,
Q.deno,
a11.id_afiliado,
a11.id_pres_prestacion,
a11.fecha,
a11.transac_rela,
a12.id_pre_prestador""")

In [33]:
transac_ref = str(df.transac_rela.tolist()).strip("[").strip("]")

df_id = querydbtopandas(f"""select transac_ref, icd as ICD from DBA.ft_cm_transac_remotas
                    where tran_tipo in ("ID")
                    and cast(fecha as date) >= date("{menor}") and cast(fecha as date) < date("{mayor}")
                    and transac_ref in ({transac_ref})""")

df_ip = df[["id_pre_prestador0","desc_pre_nombre0", "prestac_pres_prestacion", "desc_pres_prestacion", "transac_rela","fecha", "deno"]]

In [34]:
df_ip.deno = df_ip.deno.replace({None:"None"})

In [35]:
for i in validacion_0:
    df_id.loc[df_id["ICD"].str.strip() == i, "ICD" ] = 0
    
for i in validacion_1:
    df_id.loc[df_id["ICD"].str.strip() == i, "ICD" ] = 1 
    
for i in validacion_2:
    df_id.loc[df_id["ICD"].str.strip() == i, "ICD" ] = 2
    
for i in validacion_3:
    df_id.loc[df_id["ICD"].str.strip() == i, "ICD" ] = 3

In [36]:
df_id = df_id[df_id["ICD"].isin([0,1,2,3])]

## Tratamiento IP

In [37]:
for i in esp_exc:
    df_ip = df_ip[~df_ip["deno"].str.contains(i)]

#elimino todas las prestaciones menos la rmn mamaria
df_ip_rmnmamaria = df_ip[df_ip["desc_pres_prestacion"] == "RMN Mamaria"]

df_ip = df_ip[~df_ip["desc_pres_prestacion"].isin([x for x in df_ip["desc_pres_prestacion"] if 'mam' in x.lower()])]

df_ip = pd.concat([df_ip, df_ip_rmnmamaria],axis=0)

df_id = df_id.groupby("transac_ref")["ICD"].unique().reset_index()

def greatest_selection (df_id):
    for i in range(len(df_id.ICD)):
        if len(df_id.ICD[i]) <= 1:
            df_id.ICD[i] = df_id.ICD[i][0]
            
        elif len(df_id.ICD[i]) > 1:
            df_id.ICD[i].sort()
            df_id.ICD[i] = df_id.ICD[i][-1]
    return df_id

df_id = greatest_selection(df_id)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [38]:
final = pd.merge(df_ip, df_id, left_on="transac_rela",right_on= "transac_ref")

In [46]:
final.fecha = final.fecha.astype(str).str[:7]

In [50]:
final.fecha.unique()

array(['2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09',
       '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03',
       '2020-04'], dtype=object)

In [65]:
for i in final.fecha.unique():
    mes = final[(final["fecha"] == i)]
    mes = mes[["id_pre_prestador0","desc_pre_nombre0","ICD", "deno"]]
    q_transacciones= mes.groupby(["id_pre_prestador0"]).count().reset_index()[["id_pre_prestador0","desc_pre_nombre0"]].rename(columns={"desc_pre_nombre0": "Q de transacciones"})
    final_pivot = pd.pivot_table(mes, values='id_pre_prestador0', 
                   index=['id_pre_prestador0', 'desc_pre_nombre0', "deno"],
                    columns="ICD", aggfunc="size")

    final_pivot.fillna(0,inplace=True)
    final_pivot.reset_index(inplace=True)
    final_pivot = pd.merge(q_transacciones, final_pivot)

    final_pivot["Normalidad"] = (final_pivot[0] + final_pivot[1] )/ final_pivot["Q de transacciones"]

    total_esp = final_pivot.groupby("deno")["Q de transacciones"].sum().reset_index().rename(columns={"Q de transacciones": "total_esp"})

    final_pivot = pd.merge(final_pivot, total_esp)
    final_pivot["Participación"] = final_pivot["Q de transacciones"] / final_pivot["total_esp"]
    final_pivot["Participación"] = final_pivot["Participación"].mul(100).astype(float).round(2).astype(str).add('%')
    final_pivot.rename(columns={"id_pre_prestador0": "N° Prescriptor", "desc_pre_nombre0": "Prescriptor","Q de transacciones": "Total general"},inplace=True)
    final_imagenes = final_pivot[["N° Prescriptor", "Prescriptor", "deno", 0,1,2,3,  "Total general","Normalidad", "Participación"]]
    final_imagenes.columns = ["N° Prescriptor", "Prescriptor", "deno", "0","1","2","3",  "Total general","Normalidad", "Participación"]
    final_imagenes = final_imagenes.sort_values(by= "Total general", ascending= False)
    final_imagenes = final_imagenes.reset_index(drop=True).round(2)
    
    final_imagenes.to_parquet(f'datos_imagenes/{i}.gzip',compression='gzip')

In [30]:
prestador_nro = 133367

In [43]:
for i,j in zip(menores,mayores): 
    mes = final[(final["fecha"]>=datetime.datetime.strptime(i, "%Y-%m-%d")) & (final["fecha"]<datetime.datetime.strptime(j, "%Y-%m-%d"))]
    mes = mes[["id_pre_prestador0","desc_pre_nombre0","ICD", "deno"]]
    q_transacciones= mes.groupby(["id_pre_prestador0"]).count().reset_index()[["id_pre_prestador0","desc_pre_nombre0"]].rename(columns={"desc_pre_nombre0": "Q de transacciones"})
    final_pivot = pd.pivot_table(mes, values='id_pre_prestador0', 
                   index=['id_pre_prestador0', 'desc_pre_nombre0', "deno"],
                    columns="ICD", aggfunc="size")

    final_pivot.fillna(0,inplace=True)
    final_pivot.reset_index(inplace=True)
    final_pivot = pd.merge(q_transacciones, final_pivot)

    final_pivot["Normalidad"] = (final_pivot[0] + final_pivot[1] )/ final_pivot["Q de transacciones"]

    total_esp = final_pivot.groupby("deno")["Q de transacciones"].sum().reset_index().rename(columns={"Q de transacciones": "total_esp"})

    final_pivot = pd.merge(final_pivot, total_esp)
    final_pivot["Participación"] = final_pivot["Q de transacciones"] / final_pivot["total_esp"]
    final_pivot["Participación"] = final_pivot["Participación"].mul(100).astype(float).round(2).astype(str).add('%')
    final_pivot.rename(columns={"id_pre_prestador0": "N° Prescriptor", "desc_pre_nombre0": "Prescriptor","Q de transacciones": "Total general"},inplace=True)
    final_imagenes = final_pivot[["N° Prescriptor", "Prescriptor", 0,1,2,3,  "Total general","Normalidad", "Participación","deno"]]
 

    if len((final_imagenes[final_imagenes["N° Prescriptor"] == prestador_nro])) == 0:
        ar = [np.nan,np.nan]
    else:
        especialidad = final_imagenes[final_imagenes["N° Prescriptor"] == prestador_nro].values[0][-1]
        nombre = final[final["id_pre_prestador0"] == prestador_nro].iloc[0,1]
        general = final_imagenes[final_imagenes["deno"] == especialidad]["Normalidad"].mean()

        prestador = final_imagenes[final_imagenes["N° Prescriptor"] == prestador_nro].Normalidad.mean()

        ar = [prestador,general]
    
    temporal[i] = ar


In [ ]:
temporal = pd.DataFrame(columns= ["normalidad"], data= ["prestador", "especialidad"])

In [33]:
temporal = temporal.round(2)

In [44]:
temporal

,normalidad,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01
0,prestador,0.000000,0.000000,1.000000,0.142857,0.50000,0.500000,0.500000,0.200000,0.000000,0.181818,0.000000,0.000000,0.000000
1,especialidad,0.218331,0.218713,0.236514,0.204834,0.21906,0.218201,0.230165,0.240487,0.213533,0.253655,0.252344,0.229279,0.227064


In [41]:
temporal_raul

,normalidad,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01
0,prestador,0.00,0.00,1.00,0.14,0.50,0.50,0.50,0.20,0.00,0.18,0.00,0.00,0.00
1,especialidad,0.22,0.22,0.24,0.20,0.22,0.22,0.23,0.24,0.21,0.25,0.25,0.23,0.23


In [42]:
import plotly.graph_objects as go
x = temporal.columns[1:]
y_prestador = temporal[:1].values[0][1:]
y_especialidad = temporal[1:3].values[0][1:]
fig = go.Figure()
fig.add_trace(go.Scatter(x = x ,
                         y = y_prestador,
                         name = f"Prestador: {nombre}", marker_color = "#45b6fe"
                         ))
fig.add_trace(go.Scatter(x = x ,
                         y = y_especialidad,
                         name = especialidad, marker_color ="#3792cb"
                         ))
fig.update_yaxes(range=[-0.1,1.1], title="Normalidad de imágenes")
fig.update_layout(title= "",
                    yaxis_tickformat = "%")
fig.layout.legend = {"orientation": "h", "xanchor" :"center", "y":1.1, "x":0.5}
fig.show()

In [45]:
fig.write_html("imagenes.html")

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=x,
    y=temporal_prestador.iloc[6:7,1:].values[0],
    name='Primary Product',
    marker_color='#45b6fe'
))
fig.update_layout(title={"text":"Número de afiliados que calificaron por mes", 'y':0.85,'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        autosize=False,
        width=800,
        height=500, xaxis_showgrid=False, yaxis_showgrid=False)
fig.update_yaxes(title="N° de afiliados")
fig.show()